In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [2]:

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [4]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

In [14]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [15]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    api_version="2024-05-01-preview",
    model = "gpt-4o-mini",
    azure_endpoint="https://aoai-butrkhj3nqgog.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
)

In [24]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='eastus2.api.azureml.ms;ff29343b-3d73-44c7-b0ce-9f33d4a50a7a;rg-genaiops-dev;ai-project-butrkhj3nqgog',
)

In [23]:
bing_connection = project_client.connections.get(
    connection_name='kinfey-bing-search'
)

conn_id = bing_connection.id

ValueError: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.

In [18]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [19]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [20]:
bing = BingGroundingTool(connection_id=conn_id)

In [21]:
async def assistant_run() -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content="GitHub Copilot 是什么", source="user")],
            cancellation_token=CancellationToken(),
    )
#     print(response.inner_messages)
    print(response.chat_message)

In [22]:
await assistant_run()

This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_11af0j0EZ9oah3lpAh5mPqN8
Created thread, ID: thread_M9kQA3Pqbqb24uPMh82sZYAl
SMS: {'id': 'msg_jK0YMA2QoCaH7gmjHt0XnkFa', 'object': 'thread.message', 'created_at': 1737059257, 'assistant_id': None, 'thread_id': 'thread_M9kQA3Pqbqb24uPMh82sZYAl', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'GitHub Copilot 是什么', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages:GitHub Copilot是一款由GitHub和OpenAI共同开发的AI编程助手。它基于OpenAI的GPT-3模型，能够自动补全代码和提供编程建议。通过理解上下文和开发者的意图，GitHub Copilot可以生成完整的代码片段、函数或整段代码，从而提升编程效率。这个工具支持多种编程语言，包括Python、JavaScript、TypeScript、Ruby等，使其适合于广泛的编程任务。
source='assistant' models_usage=None content='GitHub Copilot是一款由GitHub和OpenAI共同开发的AI编程助手。它基于OpenAI的GPT-3模型，能够自动补全代码和提供编程建议。通过理解上下文和开发者的意图，GitHub Copilot可以生成完整的代码片段、函数或整段代码，从而提升编程效率。这个工具支持多种编程语言，包括Python、JavaScript、TypeScript、Ruby等，使其适合于广泛的编程任务。' type='TextM